In [51]:
using Pipe

get_input(file) = @pipe read(file, String) |> split(_, "\n") |> match.(r"([a-z ]+) \(contains ([a-z, ]+)\)", _) |>
    map(m -> (split(m[1], " "), strip.(split(m[2], ","))), _)

get_input (generic function with 1 method)

In [52]:
input = get_input("21-t.dat")

4-element Array{Tuple{Array{SubString{String},1},Array{SubString{String},1}},1}:
 (["mxmxvkd", "kfcds", "sqjhc", "nhms"], ["dairy", "fish"])
 (["trh", "fvjkl", "sbzzf", "mxmxvkd"], ["dairy"])
 (["sqjhc", "fvjkl"], ["soy"])
 (["sqjhc", "mxmxvkd", "sbzzf"], ["fish"])

In [53]:
getings(x) = @pipe x |> collect |> first.(_) |> vcat(_...) |> Set

getings (generic function with 1 method)

In [69]:
function find_mappings(input)
    p = Dict()
    
    for (ings, all) in input
        for a in all
            p[a] = haskey(p, a) ? intersect(p[a], Set(ings)) : Set(ings)
        end
    end
    
    p
end

function find_allergens(input)
    m = find_mappings(input)
    ings = Dict(k => [] for k in getings(input))

    for (k, v) in m
        for ing in v
            push!(ings[ing], k)
        end
    end
    
    used, final = Set(), Dict()
    
    for (k, v) in sort(filter((!)∘isempty∘last, collect(ings)), by=length∘last)
        p = filter(x -> !(x in used), v) |> first
        final[k] = p
        push!(used, p)
    end
    
    final
end

find_safe(input) = let unsafe = keys(find_allergens(input))
    @pipe getings(input) |> filter(x -> !(x in unsafe), _)
end

find_safe (generic function with 1 method)

In [63]:
find_safe(input)

Set{SubString{String}} with 4 elements:
  "sbzzf"
  "trh"
  "kfcds"
  "nhms"

In [65]:
times_appear(ings, input) =
    @pipe input |> first.(_) |> vcat(_...) |> count(x -> (x in ings), _)

times_appear (generic function with 1 method)

In [66]:
times_appear(find_safe(input), input)

5

In [58]:
@pipe find_allergens(input) |> collect |> sort(_, by=last) |> first.(_) |> join(_, ",")

"mxmxvkd,sqjhc,fvjkl"